In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!unzip -q '/content/gdrive/My Drive/HackerEarth/pass_fail.zip'

In [0]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/pass_fail/train_HK6lq50.csv')
test = pd.read_csv('/content/pass_fail/test_wF0Ps6O.csv')

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [5]:
!pip install catboost

     |████████████████████████████████| 64.8MB 68kB/s 


In [0]:
from catboost import CatBoostClassifier, Pool, cv

In [0]:
import datetime
from dateutil.relativedelta import relativedelta

In [8]:
#A function for easily LabelEncoding the columns of a dataframe
def label_encode_df(dataframe,cols):
    for col in cols:
        le=LabelEncoder()
        dataframe[col]=le.fit_transform(dataframe[col])

#Making folds for stacking
train["fold"]=0
i=1
for tr,ts in KFold(n_splits=5,shuffle=True,random_state=5).split(train):
    train.loc[list(ts),"fold"]=i
    i=i+1

#Combing train and test, and Label Encoding
comb=pd.concat([train,test],axis=0)
comb.loc[comb.trainee_engagement_rating.isna(),"trainee_engagement_rating"]=3
comb.loc[comb.age.isna(),"age"]=comb['age'].median()
# replace outliar from age column
comb['age'] = comb['age'].replace(comb['age'].max(),comb['age'].median())

# # feature extraction- creating new columns from days
# m=[]
# d=[]
# for i,row in comb.iterrows():
#     now = datetime.datetime.now()
#     td = datetime.timedelta(days=row['program_duration'])
#     five_hundred_days_ago = now - td
#     rl = relativedelta(now, five_hundred_days_ago)
#     m.append(rl.months)
#     d.append(rl.days)
# comb['month']=m
# comb['day']=d

comb.reset_index(inplace=True,drop=True)
#comb.info()

categorical_features_indices = np.where(comb.dtypes != np.float)[0]
#categorical_features_indices=[ 0  1  2  3  4  5  6  7  8  9 10 12 13 17 18]
# ['id', 'program_id', 'program_type', 'program_duration', 'test_id',
#        'test_type', 'difficulty_level', 'trainee_id', 'gender', 'education',
#        'city_tier', 'age', 'total_programs_enrolled', 'is_handicapped',
#        'trainee_engagement_rating', 'is_pass', 'fold', 'month', 'day']

cat_encoded_features=['city_tier', 'difficulty_level', 'education', 'gender',
       'is_handicapped', 'program_id','trainee_id',"test_id",
       'program_type', 'test_type']

comb_encoded=comb.copy()
#print(comb_encoded.columns)
#print(comb_encoded.info())
label_encode_df(comb_encoded,cat_encoded_features)
#print("after label encode function")

#Stacking different models (Catboost Models with different seeds)
stack=pd.DataFrame()
stack["id"]=train.id
stack["fold"]=train.fold
stack["is_pass"]=train.is_pass
model1={"model_name" : "model1", "n_estimators":2746,"model_vars" :['id', 'program_id', 'program_type', 'program_duration', 'test_id',
       'test_type', 'difficulty_level', 'trainee_id', 'gender', 'education',
       'city_tier', 'age', 'total_programs_enrolled', 'is_handicapped',
       'trainee_engagement_rating'],"cat_vars" :10,"seed" :3}
model2={"model_name" : "model2", "n_estimators":2746,"model_vars" :['id', 'program_id', 'program_type', 'program_duration', 'test_id',
       'test_type', 'difficulty_level', 'trainee_id', 'gender', 'education',
       'city_tier', 'age', 'total_programs_enrolled', 'is_handicapped',
       'trainee_engagement_rating'],"cat_vars" :10,"seed":12}
model3={"model_name" : "model3", "n_estimators":2746,"model_vars" :['id', 'program_id', 'program_type', 'program_duration', 'test_id',
       'test_type', 'difficulty_level', 'trainee_id', 'gender', 'education',
       'city_tier', 'age', 'total_programs_enrolled', 'is_handicapped',
       'trainee_engagement_rating'],"cat_vars" :10, "seed":30}
model4={"model_name" : "model4", "n_estimators":2746,"model_vars" :['id', 'program_id', 'program_type', 'program_duration', 'test_id',
       'test_type', 'difficulty_level', 'trainee_id', 'gender', 'education',
       'city_tier', 'age', 'total_programs_enrolled', 'is_handicapped',
       'trainee_engagement_rating'],"cat_vars" :10, "seed":66}
model5={"model_name" : "model5", "n_estimators":2746,"model_vars" :['id', 'program_id', 'program_type', 'program_duration', 'test_id',
       'test_type', 'difficulty_level', 'trainee_id', 'gender', 'education',
       'city_tier', 'age', 'total_programs_enrolled', 'is_handicapped',
       'trainee_engagement_rating'],"cat_vars" :10, "seed":72}
models=[model1,model2,model3,model4,model5]

#Running the models on 4 folds and predicting on the 5th
#print("before for loop")
for model in models:
    stack[model["model_name"]]=0
    for i in range(1,6):
        print(model["model_name"])
        comb_encoded["dataset"]="train"
        len_train=73147
        comb_encoded.loc[73147:,"dataset"]="test"
        comb_encoded.loc[comb_encoded.fold==i,"dataset"]="valid"
        y=comb_encoded.loc[comb_encoded.dataset=="train","is_pass"].values
        y_test=comb_encoded.loc[comb_encoded.dataset=="valid","is_pass"].values
        x=comb_encoded.loc[comb_encoded.dataset=="train",model["model_vars"]].values
        x_test=comb_encoded.loc[comb_encoded.dataset=="valid",model["model_vars"]].values
        cat_model=CatBoostClassifier(eval_metric="AUC",n_estimators=model["n_estimators"],random_state=model["seed"])
        cat_model.fit(x,y,cat_features=list(range(0,model["cat_vars"])),verbose=False)
        stack.loc[stack.fold==i,model["model_name"]]=cat_model.predict_proba(comb_encoded.loc[comb_encoded.dataset=="valid",model["model_vars"]].values)[:,1]

#Running the base models and the whole train set and predicitng for test set
stack_test=pd.DataFrame()
stack_test["id"]=test.id
for model in models:
    stack_test[model["model_name"]]=0
    print(model["model_name"])
    comb_encoded["dataset"]="train"
    len_train=73147
    comb_encoded.loc[73147:,"dataset"]="test"
    #comb_encoded.loc[comb_encoded.fold==i,"dataset"]="valid"
    y=comb_encoded.loc[comb_encoded.dataset=="train","is_pass"].values
    #y_test=comb_encoded.loc[comb_encoded.dataset=="valid","is_pass"].values
    x=comb_encoded.loc[comb_encoded.dataset=="train",model["model_vars"]].values
    #x_test=comb_encoded.loc[comb_encoded.dataset=="valid",model["model_vars"]].values
    cat_model=CatBoostClassifier(eval_metric="AUC",n_estimators=model["n_estimators"],random_state=model["seed"])
    cat_model.fit(x,y,cat_features=list(range(0,model["cat_vars"])),verbose=False)
    stack_test[model["model_name"]]=cat_model.predict_proba(comb_encoded.loc[comb_encoded.dataset=="test",model["model_vars"]].values)[:,1]


model1
model1
model1
model1
model1
model2
model2
model2
model2
model2
model3
model3
model3
model3
model3
model4
model4
model4
model4
model4
model5
model5
model5
model5
model5
model1
model2
model3
model4
model5


In [0]:
# import lightgbm as lgb

In [0]:
# parameters = {
#     'application': 'binary',
#     'objective': 'binary',
#     'metric': 'auc',
#     'is_unbalance': 'true',
#     'boosting': 'gbdt',
#     'verbose': 0
# }

In [0]:
#Running a stacked model on the 3 base models and making final predications
lr_model=LogisticRegression()
lr_model.fit(X=stack[["model1","model2","model3","model4","model5"]],y=stack.is_pass) 
stack_test["is_pass"]=lr_model.predict_proba(X=stack_test[["model1","model2","model3","model4","model5"]])[:,1]
stack_test[["id","is_pass"]].to_csv("sub_stacked_updated_5models72.csv", index=False)

In [0]:
# score in lb: 0.8368729349
# with 10 fold score is 0.83358966816
# stacking with lgbm scores 0.832344998
# with custom catbbost 0.836128879447
# 4 models stacking give me best score 0.83735928
# 5 models stacking decreased dcore to 0.8371855479